1. KcELECTRA
- https://www.dinolabs.ai/400

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import csv
import requests
from collections import Counter

import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda:0


## 전처리

#### 불용어 처리
- 불용어 리스트는 https://www.ranks.nl/stopwords/korean 에서 가져옴

**교수님 피드백: 셀프어텐션을 사용하는 BERT기반 모델의 경우 조사나 접속어 같은 것들을 제거해주면 주어, 목적어, 동사 등의 구분이 되지 않아 오히려 성능이 떨어질 우려가 있음**

In [ ]:
# 데이터셋 1, 2중 하나 사용
# merged_df = pd.read_csv('/content/drive/MyDrive/CUAI 컨퍼런스/data/merged_df.csv')
merged_df = pd.read_csv('/content/drive/MyDrive/CUAI 컨퍼런스/data/merged_df(감정 분류).csv')[['content', 'label']]

In [ ]:
# # 문자열 아닌 데이터 모두 제거
# merged_df['content'] = [content for content in merged_df['content'] if type(content) is str]

In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','것','이다','게','에서','거','로','수','에게','요']

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 35.6 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm

# KoNLPy의 Okt 형태소 분석기를 사용하여 불용어 제거
okt = Okt()

def remove_stopwords(text):
    words = okt.pos(text, stem=True)  # 형태소 분석, (stem=True -> 기본형?으로 나옴)
    filtered_words = [word for word, pos in words if word not in stopwords]  # 불용어 제거
    return ' '.join(filtered_words)


# 'content' 열에 있는 각 텍스트에 대해 불용어 제거 수행
tqdm.pandas()
merged_df['content'] = merged_df['content'].progress_apply(remove_stopwords)

100%|██████████| 45999/45999 [01:36<00:00, 478.76it/s]


#### 텍스트 정규화

In [ ]:
import re
from tqdm import tqdm

def text_normalization(text):
    # 소문자 변환
    text = text.lower()

    # 숫자 제거
    text = re.sub(r'\d+', '', text)

    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)

    return text

tqdm.pandas()
merged_df['content'] = merged_df['content'].progress_apply(text_normalization)

100%|██████████| 45999/45999 [00:00<00:00, 222717.33it/s]


#### 반복 표현 제거(ex. ㅋㅋㅋㅋㅋㅋ,ㅎㅎㅎㅎㅎ)

In [ ]:
def replace_repeated_chars(text):
    # 두 번 이상 반복되는 글자(예: ㅋㅋㅋ, ㅎㅎㅎ)를 찾아 해당 글자의 두 번 반복으로 치환
    pattern = re.compile(r'(.)\1{1,}', re.DOTALL)

    # 정규식에 맞게 두 번 이상 반복되는 글자를 찾아 치환
    text = pattern.sub(r'\1\1', text)

    return text

# 'content' 열에 있는 각 텍스트에 대해 두 번 이상 반복되는 글자를 찾아 치환
tqdm.pandas()
merged_df['content'] = merged_df['content'].progress_apply(replace_repeated_chars)

100%|██████████| 45999/45999 [00:00<00:00, 97871.70it/s] 


In [ ]:
df_length = merged_df['content'].astype(str).apply(len)
print('글 길이 최댓값: {}'.format(np.max(df_length)))
print('글 길이 최솟값: {}'.format(np.min(df_length)))
print('글 길이 제3사분위: {}'.format(np.percentile(df_length,95)))

글 길이 최댓값: 123
글 길이 최솟값: 0
글 길이 제3사분위: 50.0


In [ ]:
merged_df = merged_df[merged_df['content'].str.len() > 5]
merged_df.shape

(45626, 2)

### 학습, 검증 데이터 분리

In [ ]:
merged_df.drop_duplicates(subset=['content'], inplace=True, ignore_index=True) # 중복 제거

In [ ]:
train_data = merged_df.sample(frac=0.8, random_state=42)
val_data = merged_df.drop(train_data.index)

len(train_data), len(val_data)

(36317, 9079)

In [ ]:
print(train_data['label'].value_counts())
print(val_data['label'].value_counts())

1    18336
0    17981
Name: label, dtype: int64
1    4644
0    4435
Name: label, dtype: int64


## 토큰화

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base-v2022", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/450k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_labels = train_data['label'].values
train_texts = train_data['content']

val_labels = val_data['label'].values
val_texts = val_data['content']

In [ ]:
train_endocings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128,
                            return_tensors='pt') # tensor 형태로 return

val_endocings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128,
                            return_tensors='pt') # tensor 형태로 return

## 학습 데이터셋 생성

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}


    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset = Dataset(train_endocings, train_labels)
val_dataset = Dataset(val_endocings, val_labels)

## 학습

In [ ]:
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
training_args = TrainingArguments(
    output_dir='/tmp/results',
    evaluation_strategy = "epoch",
    num_train_epochs=7,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir='/tmp/logs',
    logging_steps=500,
    save_total_limit=2
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.251200,0.303213,0.901311,0.907838,0.869043,0.950258
2,0.198200,0.297838,0.910783,0.908763,0.952763,0.868648
3,0.152100,0.305388,0.919925,0.922138,0.917324,0.927003
4,0.085600,0.313437,0.918053,0.919463,0.924467,0.914513
5,0.065500,0.393198,0.917502,0.920175,0.910952,0.929587
6,0.047100,0.473301,0.913757,0.916640,0.906507,0.927003
7,0.024200,0.517752,0.915299,0.918616,0.903226,0.934539


TrainOutput(global_step=15890, training_loss=0.11983227390699824, metrics={'train_runtime': 3221.8397, 'train_samples_per_second': 78.905, 'train_steps_per_second': 4.932, 'total_flos': 7054575755192280.0, 'train_loss': 0.11983227390699824, 'epoch': 7.0})

## 학습된 모델 저장 및 불러오기
- config.json: 모델의 구조와 설정이 포함된 JSON 파일입니다. 모델을 초기화할 때 사용됩니다.
- training_args.bin (옵션): 모델을 훈련할 때 사용된 훈련 설정이 저장된 이진 파일입니다. 훈련된 모델을 재사용하거나 파인튜닝할 때 사용됩니다.
- model.safetensors: 이 파일은 Transformer 모델의 내부 상태를 저장하는 것으로 보입니다. 하지만 표준적인 Transformer 모델 저장 파일이 아니며, 특별한 상황에서 생성된 것일 수 있습니다.

In [ ]:
# trainer.save_model('/content/drive/MyDrive/CUAI 컨퍼런스/result/KcELECTRA(데이터셋2,0.89)')

In [ ]:
# from transformers import ElectraTokenizer, ElectraForSequenceClassification

# # 'saved_model_path'는 Electra 모델이 저장된 디렉토리를 나타냅니다.
# saved_model_path = '/content/drive/MyDrive/CUAI 컨퍼런스/result/KcELECTRA(감정으로 분류, 0.87)'

# # 토크나이저와 모델 불러오기
# model = ElectraForSequenceClassification.from_pretrained(saved_model_path)

## 검증

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/CUAI 컨퍼런스/data/test_data.csv')
test_data['content'] = test_data['title'] + ' ' + test_data['content']

# 학습데이터와 같은 전처리 적용
test_data['content'] = test_data['content'].progress_apply(remove_stopwords)
test_data['content'] = test_data['content'].progress_apply(text_normalization)
test_data['content'] = test_data['content'].progress_apply(replace_repeated_chars)

100%|██████████| 390/390 [00:00<00:00, 46980.83it/s]


In [ ]:
predicted_list = []

for text in test_data['content']:

    # 토큰화
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # 입력 데이터를 모델이 동작 중인 디바이스로 이동
    device = model.device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 문장별 logit을 합해서 클래스 분류(나중에 softmax 적용시 logit을 합할지 평균으로 할지에 따라 값이 바뀌니 주의)
    logits_sum = torch.sum(outputs.logits, dim=0)
    predicted_class = torch.argmax(logits_sum, dim=-1).cpu().numpy()

    # 예측된 클래스를 리스트에 추가
    predicted_list.append(predicted_class)

# 예측된 클래스를 DataFrame에 추가
test_data['predicted_class'] = predicted_list
test_data.head()

,title,content,label,predicted_class
0,아빠가 돌아가셨는데,아빠 돌아가다 시체 인생 처음 으로 보다 체감 안 돼다 아무리 보다 자다 거 같다 ...,1,1
1,좀 더러울 수도 있는데 혹시 이런 사람 있어?,좀 더럽다 수도 있다 혹시 이렇다 사람 있다 배 가스 너무 많이 차 ㅠ 뭘 먹다 ...,1,1
2,사람만나는게,사람 만나다 재밌다 않다 일 같다 일 하다 싫다 숙제 꾸역꾸역 하다 느낌,1,1
3,힘들어할 자격이 있을까,힘들다 하다 자격 있다 힘들다 자격 조차 없다 난 어쩌지,1,1
4,인간관계,인간관계 친구 라고 생각 하고 있다 뒤 에서 시기 질투 하다 있다 ㅋㅋ 여행 같이 ...,1,1


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
test_data['predicted_class'] = test_data['predicted_class'].astype(int)

predicted_labels = test_data['predicted_class']
true_labels = test_data['label']

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)

# 재현율 계산
recall = recall_score(true_labels, predicted_labels)

# 정밀도 계산
precision = precision_score(true_labels, predicted_labels)

# F1 score 계산
f1 = f1_score(true_labels, predicted_labels)

# 결과 출력
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.8410
Recall: 0.9289
Precision: 0.7922
F1 Score: 0.8551


### 2. 한 문장에 대해 예측

In [ ]:
def predict(sent):
  model.eval()
  tokenized_sent = tokenizer(sent, truncation=True, padding=True, max_length=128,
                            return_tensors='pt')

  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )

    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    if result == 0:
      result = '우울증X'
    elif result == 1:
      result = '우울증'
    return result

sent = '너무 슬퍼'
predict(sent)